# **CUSTOMER SEGMENTATION WITH RFM**

## Business Problem
FLO, an online shoe store, wants to segment its customers and determine marketing strategies based on these segments. For this purpose, customer behaviors will be defined, and groups will be formed based on clustering in these behaviors.

## Data Set
The dataset consists of information derived from the past shopping behaviors of customers who made purchases from FLO in the years 2020-2021 as OmniChannel customers, meaning they shopped both online and offline.

## Variables
- **master_id**: Unique customer number id
- **order_channel** : Which channel is used (Android, ios, Desktop, Mobile, Offline)
- **last_order_channel** : Last order channel
- **first_order_date** : the first order date of customer
- **last_order_date** : the last order date of customer
- **last_order_date_online** : the last order date of customer in online platform
- **last_order_date_offline** : the last order date of customer in offline platform
- **order_num_total_ever_online** : the total number of online order of customer
- **order_num_total_ever_offline** : the total number of offline order of customer
- **customer_value_total_ever_offline** : Total fee paid by the customer for offline purchases
- **customer_value_total_ever_online** : Total fee paid by the customer for online purchases
- **interested_in_categories_12** : List of categories the customer has shopped in the last 12 months

# 1. Understanding & Preparing Data

In [1]:
import pandas as pd
import datetime as dt
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.width',1000)

In [2]:
df_ = pd.read_csv("/kaggle/input/flo-customer-data/flo_data_20k.csv")
df = df_.copy()

df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.00000,1.00000,139.99000,799.38000,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.00000,2.00000,159.97000,1853.58000,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.00000,2.00000,189.97000,395.35000,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.00000,1.00000,39.99000,81.98000,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.00000,1.00000,49.99000,159.99000,[AKTIFSPOR]


In [3]:
df.dtypes

master_id                             object
order_channel                         object
last_order_channel                    object
first_order_date                      object
last_order_date                       object
last_order_date_online                object
last_order_date_offline               object
order_num_total_ever_online          float64
order_num_total_ever_offline         float64
customer_value_total_ever_offline    float64
customer_value_total_ever_online     float64
interested_in_categories_12           object
dtype: object

In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.00000,3.11085,4.22565,1.00000,1.00000,2.00000,4.00000,200.00000
order_num_total_ever_offline,19945.00000,1.91391,2.06288,1.00000,1.00000,1.00000,2.00000,109.00000
customer_value_total_ever_offline,19945.00000,253.92260,301.53285,10.00000,99.99000,179.98000,319.97000,18119.14000
customer_value_total_ever_online,19945.00000,497.32169,832.60189,12.99000,149.98000,286.46000,578.44000,45220.13000


In [5]:
df.isnull().sum()

master_id                            0
order_channel                        0
last_order_channel                   0
first_order_date                     0
last_order_date                      0
last_order_date_online               0
last_order_date_offline              0
order_num_total_ever_online          0
order_num_total_ever_offline         0
customer_value_total_ever_offline    0
customer_value_total_ever_online     0
interested_in_categories_12          0
dtype: int64

In [6]:
#Creating total order number and total customer value variables
df["total_order_num"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["total_customer_value"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]
df[["total_order_num", "total_customer_value"]].head(10)

,total_order_num,total_customer_value
0,5.00000,939.37000
1,21.00000,2013.55000
2,5.00000,585.32000
3,2.00000,121.97000
4,2.00000,209.98000
5,3.00000,200.86000
6,4.00000,375.93000
7,2.00000,163.63000
8,5.00000,1054.69000
9,2.00000,165.96000


In [7]:
# Convert date variables(object) to date type
date_columns = df.columns[df.columns.str.contains("date")]
df[date_columns] = df[date_columns].apply(pd.to_datetime)
df.dtypes

master_id                                    object
order_channel                                object
last_order_channel                           object
first_order_date                     datetime64[ns]
last_order_date                      datetime64[ns]
last_order_date_online               datetime64[ns]
last_order_date_offline              datetime64[ns]
order_num_total_ever_online                 float64
order_num_total_ever_offline                float64
customer_value_total_ever_offline           float64
customer_value_total_ever_online            float64
interested_in_categories_12                  object
total_order_num                             float64
total_customer_value                        float64
dtype: object

In [8]:
df.groupby("order_channel").agg({"master_id": "count",
                                            "total_order_num": "sum",
                                            "total_customer_value": "sum"})

,master_id,total_order_num,total_customer_value
order_channel,,,
Android App,9495,52269.00000,7819062.76000
Desktop,2735,10920.00000,1610321.46000
Ios App,2833,15351.00000,2525999.93000
Mobile,4882,21679.00000,3028183.16000


In [9]:
# Top 10 customers with the most revenue
df.sort_values("total_customer_value", ascending=False).head(10)

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,total_order_num,total_customer_value
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.00000,2.00000,684.97000,45220.13000,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",202.00000,45905.10000
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,Android App,Android App,2018-08-06,2021-02-23,2021-02-23,2020-07-06,67.00000,1.00000,130.49000,36687.80000,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",68.00000,36818.29000
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,Ios App,Offline,2014-01-14,2021-05-18,2021-01-30,2021-05-18,81.00000,1.00000,1263.76000,32654.34000,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",82.00000,33918.10000
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,Ios App,Offline,2021-03-01,2021-04-13,2021-03-18,2021-04-13,10.00000,1.00000,538.94000,30688.47000,"[ERKEK, KADIN, AKTIFSPOR]",11.00000,31227.41000
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,Android App,Offline,2021-04-07,2021-04-27,2021-04-07,2021-04-27,1.00000,3.00000,18119.14000,2587.20000,[AKTIFSPOR],4.00000,20706.34000
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,Desktop,Desktop,2020-02-16,2021-04-30,2021-04-30,2020-12-18,66.00000,4.00000,843.68000,17599.89000,"[ERKEK, KADIN, AKTIFSPOR]",70.00000,18443.57000
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,Ios App,Ios App,2017-05-10,2021-04-13,2021-04-13,2019-08-15,69.00000,1.00000,82.48000,16836.09000,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",70.00000,16918.57000
163,fef57ffa-aae6-11e9-a2fc-000d3a38a36f,Mobile,Desktop,2016-11-08,2021-05-12,2021-05-12,2020-07-09,36.00000,1.00000,180.73000,12545.37000,"[ERKEK, AKTIFSPOR]",37.00000,12726.10000
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,Android App,Android App,2013-02-21,2021-05-09,2021-05-09,2020-01-25,130.00000,1.00000,49.99000,12232.25000,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",131.00000,12282.24000
18767,fc0ce7a4-9d87-11e9-9897-000d3a38a36f,Desktop,Desktop,2018-11-24,2020-11-11,2020-11-11,2019-12-06,18.00000,2.00000,64.97000,12038.18000,"[ERKEK, KADIN]",20.00000,12103.15000


In [10]:
# Top 10 customers with the most frequency
df.sort_values("total_order_num", ascending=False).head(10)

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,total_order_num,total_customer_value
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.00000,2.00000,684.97000,45220.13000,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",202.00000,45905.10000
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,Android App,Android App,2013-02-21,2021-05-09,2021-05-09,2020-01-25,130.00000,1.00000,49.99000,12232.25000,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",131.00000,12282.24000
8783,a57f4302-b1a8-11e9-89fa-000d3a38a36f,Android App,Offline,2019-08-07,2020-11-04,2020-09-07,2020-11-04,2.00000,109.00000,10239.46000,143.98000,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",111.00000,10383.44000
2619,fdbe8304-a7ab-11e9-a2fc-000d3a38a36f,Android App,Offline,2018-10-18,2020-06-30,2018-10-18,2020-06-30,1.00000,87.00000,8432.25000,139.98000,[ERKEK],88.00000,8572.23000
6322,329968c6-a0e2-11e9-a2fc-000d3a38a36f,Ios App,Ios App,2019-02-14,2021-04-05,2021-04-05,2020-02-17,2.00000,81.00000,3997.55000,242.81000,[ERKEK],83.00000,4240.36000
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,Ios App,Offline,2014-01-14,2021-05-18,2021-01-30,2021-05-18,81.00000,1.00000,1263.76000,32654.34000,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",82.00000,33918.10000
9347,44d032ee-a0d4-11e9-a2fc-000d3a38a36f,Mobile,Mobile,2019-02-11,2021-02-11,2021-02-11,2020-12-24,3.00000,74.00000,4640.77000,543.28000,"[KADIN, AKTIFSPOR]",77.00000,5184.05000
10954,b27e241a-a901-11e9-a2fc-000d3a38a36f,Mobile,Mobile,2015-09-12,2021-04-01,2021-04-01,2019-10-08,72.00000,3.00000,292.93000,5004.95000,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",75.00000,5297.88000
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,Ios App,Ios App,2017-05-10,2021-04-13,2021-04-13,2019-08-15,69.00000,1.00000,82.48000,16836.09000,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",70.00000,16918.57000
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,Desktop,Desktop,2020-02-16,2021-04-30,2021-04-30,2020-12-18,66.00000,4.00000,843.68000,17599.89000,"[ERKEK, KADIN, AKTIFSPOR]",70.00000,18443.57000


# 2. Calculating RFM Metrics

In [11]:
df["last_order_date"].max() # 2021-05-30
analysis_date = dt.datetime(2021, 6, 1) #to analyze as if it were up to date

rfm = pd.DataFrame()
rfm["customer_id"] = df["master_id"]
rfm["recency"] = (analysis_date - df["last_order_date"]).astype("timedelta64[D]")
rfm["frequency"] = df["total_order_num"]
rfm["monetary"] = df["total_customer_value"]

rfm.head()

,customer_id,recency,frequency,monetary
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95.00000,5.00000,939.37000
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105.00000,21.00000,2013.55000
2,69b69676-1a40-11ea-941b-000d3a38a36f,186.00000,5.00000,585.32000
3,1854e56c-491f-11eb-806e-000d3a38a36f,135.00000,2.00000,121.97000
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,86.00000,2.00000,209.98000


In [12]:
# Calculating RFM Scores
rfm["recency_score"] = pd.qcut(rfm["recency"], q=5, labels=[5, 4, 3, 2, 1]) #the lowest recency means the highest value.
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), q=5, labels=[1, 2, 3, 4, 5])
rfm["monetary_score"] = pd.qcut(rfm["monetary"], q=5, labels=[1, 2, 3, 4, 5])
rfm.head()

,customer_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95.00000,5.00000,939.37000,3,4,4
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105.00000,21.00000,2013.55000,3,5,5
2,69b69676-1a40-11ea-941b-000d3a38a36f,186.00000,5.00000,585.32000,2,4,3
3,1854e56c-491f-11eb-806e-000d3a38a36f,135.00000,2.00000,121.97000,3,1,1
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,86.00000,2.00000,209.98000,3,1,1


In [13]:
# Creating RF Score
rfm["RF_SCORE"] = (rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str))

In [14]:
# Defining Segmentation of RF Scores
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}
rfm["segment"] = rfm["RF_SCORE"].replace(seg_map, regex=True)
rfm.head()

,customer_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95.00000,5.00000,939.37000,3,4,4,34,loyal_customers
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105.00000,21.00000,2013.55000,3,5,5,35,loyal_customers
2,69b69676-1a40-11ea-941b-000d3a38a36f,186.00000,5.00000,585.32000,2,4,3,24,at_Risk
3,1854e56c-491f-11eb-806e-000d3a38a36f,135.00000,2.00000,121.97000,3,1,1,31,about_to_sleep
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,86.00000,2.00000,209.98000,3,1,1,31,about_to_sleep


# 3. Case Studies

## Case1
[ENG]
FLO is incorporating a new women's footwear brand. Product prices of the brand it includes general customer above their preferences. Therefore, we are specifically engaged in the promotion of the brand and with customers of a profile who will be interested in product sales. it is desirable to get in touch. Shop from loyal customers (champions, loyal_customers) and from the women's category people who make customers to be contacted privately. Save the IDs of the customers in the appropriate profile to the csv file.

[TR]
FLO bünyesine yeni bir kadın ayakkabı markası dahil ediyor. Dahil ettiği markanın ürün fiyatları genel müşteri
tercihlerinin üstünde. Bu nedenle markanın tanıtımı ve ürün satışları için ilgilenecek profildeki müşterilerle özel olarak
iletişime geçmek isteniliyor. Sadık müşterilerinden(champions, loyal_customers) ve kadın kategorisinden alışveriş
yapan kişiler özel olarak iletişim kurulacak müşteriler. Bu müşterilerin idlerini bulup csv olarak kaydediniz.

In [15]:
segment_index = rfm.loc[(rfm["segment"] == "champions") | (rfm["segment"] == "loyal_customers"), "customer_id"]
target_customer_ids = df[df["master_id"].isin(segment_index) & (df["interested_in_categories_12"].str.contains("KADIN"))]["master_id"]
target_customer_ids.to_csv("yeni_marka_hedef_müşteri_id.csv", index=False)
target_customer_ids


0        cc294636-19f0-11eb-8d74-000d3a38a36f
1        f431bd5a-ab7b-11e9-a2fc-000d3a38a36f
8        cfbda69e-5b4f-11ea-aca7-000d3a38a36f
11       c2e15af2-9eed-11e9-9897-000d3a38a36f
33       fb840306-1219-11ea-a001-000d3a38a36f
                         ...                 
19912    f63aba0e-41d8-11ea-96d9-000d3a38a36f
19913    81d0da04-a53e-11e9-a2fc-000d3a38a36f
19914    825e6b00-ab40-11e9-a2fc-000d3a38a36f
19917    f8c471c8-2596-11eb-81e9-000d3a38a36f
19926    7e1c15be-6298-11ea-9861-000d3a38a36f
Name: master_id, Length: 2497, dtype: object

## Case2
[ENG]Up to 40% discount is planned on Men's and Children's products. In the past, you have been interested in categories related to this discount customers who are good customers but haven't shopped for a long time, who shouldn't be lost, those who are asleep, and new incoming customers are specifically targeted. Save the IDs of the customers in the appropriate profile to the csv file.

[TR]Erkek ve Çocuk ürünlerinde %40'a yakın indirim planlanmaktadır. Bu indirimle ilgili kategorilerle ilgilenen geçmişte
iyi müşteri olan ama uzun süredir alışveriş yapmayan kaybedilmemesi gereken müşteriler, uykuda olanlar ve yeni
gelen müşteriler özel olarak hedef alınmak isteniyor. Uygun profildeki müşterilerin id'lerini csv dosyasına kaydediniz.

In [16]:
segment_index_2 = rfm.loc[(rfm["segment"].values == "about_to_sleep") | (rfm["segment"].values == "new_customers"), "customer_id"]
target_customer_ids2 = df[df["master_id"].isin(segment_index_2) & df["interested_in_categories_12"].str.contains("ERKEK", "COCUK")]["master_id"]
target_customer_ids2.to_csv("indirim_hedef_müşteri_ids.csv", index=False)
target_customer_ids2

34       1acf7870-87af-11ea-ace9-000d3a38a36f
56       92a334ee-58f2-11eb-9e65-000d3a38a36f
78       56ddd00a-aff6-11e9-9757-000d3a38a36f
126      71976318-2aa0-11eb-b280-000d3a38a36f
142      d3fea560-aced-11ea-b736-000d3a38a36f
                         ...                 
19727    3c2bf5b0-5bb4-11ea-81c0-000d3a38a36f
19760    ec238d4a-242b-11ea-a30f-000d3a38a36f
19823    4eb38320-a691-11e9-a2fc-000d3a38a36f
19832    c4b363d4-c6c4-11ea-9d14-000d3a38a36f
19935    b514d468-5b52-11ea-85ca-000d3a38a36f
Name: master_id, Length: 531, dtype: object

# 4. Defining a function that will do the whole process

In [17]:
def rfm_function(dataframe):
    
    # Data preprocessing
    dataframe["total_order_num"] = dataframe["order_num_total_ever_online"] + dataframe["order_num_total_ever_offline"]
    dataframe["total_customer_value"] = dataframe["customer_value_total_ever_offline"] + dataframe["customer_value_total_ever_online"]
    dataframe[["total_order_num", "total_customer_value"]].head(10)
    
    date_columns = dataframe.columns[dataframe.columns.str.contains("date")]
    dataframe[date_columns] = dataframe[date_columns].apply(pd.to_datetime)
    dataframe.dtypes
    
    dataframe.groupby("order_channel").agg({"master_id": "count",
                                            "total_order_num": "sum",
                                            "total_customer_value": "sum"})
    # Calculating RFM Metrics
    dataframe["last_order_date"].max() # 2021-05-30
    analysis_date = dt.datetime(2021, 6, 1)
    rfm = pd.DataFrame()
    rfm["customer_id"] = dataframe["master_id"]
    rfm["recency"] = (analysis_date - dataframe["last_order_date"]).astype('timedelta64[D]')
    rfm["frequency"] = dataframe["total_order_num"]
    rfm["monetary"] = dataframe["total_customer_value"]
    
    
    
    # Calculating RFM Scores
    rfm["recency_score"] = pd.qcut(rfm["recency"], q=5, labels=[5, 4, 3, 2, 1]) #the lowest recency means the highest value.
    rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), q=5, labels=[1, 2, 3, 4, 5])
    rfm["monetary_score"] = pd.qcut(rfm["monetary"], q=5, labels=[1, 2, 3, 4, 5])
    rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str) + rfm['monetary_score'].astype(str))
    
    
    
    # Creating RF Score
    rfm["RF_SCORE"] = (rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str))
    
    
    # Defining Segmentation of RF Scores
    seg_map = {
        r'[1-2][1-2]': 'hibernating',
        r'[1-2][3-4]': 'at_Risk',
        r'[1-2]5': 'cant_loose',
        r'3[1-2]': 'about_to_sleep',
        r'33': 'need_attention',
        r'[3-4][4-5]': 'loyal_customers',
        r'41': 'promising',
        r'51': 'new_customers',
        r'[4-5][2-3]': 'potential_loyalists',
        r'5[4-5]': 'champions'
    }
    rfm["segment"] = rfm["RF_SCORE"].replace(seg_map, regex=True)

    
    return rfm[["customer_id", "recency","frequency","monetary","RF_SCORE","RFM_SCORE","segment"]]

In [18]:
rfm_df = rfm_function(df)
rfm_df

,customer_id,recency,frequency,monetary,RF_SCORE,RFM_SCORE,segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95.00000,5.00000,939.37000,34,344,loyal_customers
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105.00000,21.00000,2013.55000,35,355,loyal_customers
2,69b69676-1a40-11ea-941b-000d3a38a36f,186.00000,5.00000,585.32000,24,243,at_Risk
3,1854e56c-491f-11eb-806e-000d3a38a36f,135.00000,2.00000,121.97000,31,311,about_to_sleep
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,86.00000,2.00000,209.98000,31,311,about_to_sleep
...,...,...,...,...,...,...,...
19940,727e2b6e-ddd4-11e9-a848-000d3a38a36f,331.00000,3.00000,401.96000,13,132,at_Risk
19941,25cd53d4-61bf-11ea-8dd8-000d3a38a36f,161.00000,2.00000,390.47000,22,222,hibernating
19942,8aea4c2a-d6fc-11e9-93bc-000d3a38a36f,8.00000,3.00000,632.94000,53,533,potential_loyalists
19943,e50bb46c-ff30-11e9-a5e8-000d3a38a36f,108.00000,6.00000,1009.77000,34,344,loyal_customers
